In [ ]:
%matplotlib inline
from datetime import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from IPython.display import display
import ipywidgets as widgets
import matplotlib.pyplot as plt
from metpy.units import units
import numpy as np
import metpy.calc as mpcalc
from metpy.plots.ctables import colortables
from matplotlib import rcParams, colors
import xarray as xr


In [ ]:
def getdata(year=2001,month=4, day=12, hour=12, save=False):
    date = datetime(year, month, day, hour)

    # Choose a dataset...
    
    # Current Data
    # ds = xr.open_dataset('https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/GFS/Global_0p5deg_ana/TP')
    
    # UCAR RDA Archive - Global 1.0 deg Data from August 1, 1999 to current
    # if datetime(1999, 7, 30, 18) <= date <= datetime(2007, 12, 6, 0):
    #     ds = xr.open_dataset('https://tds.gdex.ucar.edu/thredds/dodsC/files/g/d083002/grib1/'
    #                          f'{date:%Y}/{date:%Y.%m}/fnl_{date:%Y%m%d_%H}_00.grib1')
    # elif date > datetime(2007, 12, 6, 0):
    #     ds = xr.open_dataset('https://tds.gdex.ucar.edu/thredds/dodsC/files/g/d083002/grib2/'
    #                          f'{date:%Y}/{date:%Y.%m}/fnl_{date:%Y%m%d_%H}_00.grib2')

    # UCAR RDA Archive - Global 0.25 deg Data from 2015 to current
    ds = xr.open_dataset('https://tds.gdex.ucar.edu/thredds/dodsC/files/g/d083003/'
                           f'{date:%Y}/{date:%Y%m}/gdas1.fnl0p25.{date:%Y%m%d%H}.f00.grib2')

    # Local File
    #ds = xr.open_dataset(f'groundhogs_day_blizzard/GFS_{date:%Y%m%d}_{date:%H}00.nc')

    # ACCESS DATA
    data = ds.metpy.sel(lat=slice(60, 10), lon=slice(360-160, 360-50))

    hght_data = data.Geopotential_height_isobaric
    uwnd_data = data['u-component_of_wind_isobaric']
    vwnd_data = data['v-component_of_wind_isobaric']
    tmpk_data = data.Temperature_isobaric
    relh_data = data.Relative_humidity_isobaric
    
    # Pull out the lat and lon data
    lat = data.lat.values
    lon = data.lon.values
    lons, lats = np.meshgrid(lon, lat)

    # Set up map for plotting
    plotcrs = ccrs.LambertConformal(central_latitude=45., central_longitude=-100.,
                                    standard_parallels=[30, 60])

    datacrs = ccrs.PlateCarree()

    tlatslons = plotcrs.transform_points(datacrs, lons, lats)
    tlon = tlatslons[:, :, 0]
    tlat = tlatslons[:, :, 1]

    # Set smoothing iterations
    smooth = 2

    # Subset data and smooth
    hght_1000 = mpcalc.smooth_n_point(hght_data.metpy.sel(time=date, vertical=1000*units.hPa), 9, smooth)
    hght_500 = mpcalc.smooth_n_point(hght_data.metpy.sel(time=date, vertical=500*units.hPa), 9, smooth)
    uwnd_500 = mpcalc.smooth_n_point(uwnd_data.metpy.sel(time=date, vertical=500*units.hPa), 9, smooth)
    vwnd_500 = mpcalc.smooth_n_point(vwnd_data.metpy.sel(time=date, vertical=500*units.hPa), 9, smooth)
    
    hght_850 = mpcalc.smooth_n_point(hght_data.metpy.sel(time=date, vertical=850*units.hPa), 9, smooth)
    uwnd_850 = mpcalc.smooth_n_point(uwnd_data.metpy.sel(time=date, vertical=850*units.hPa), 9, smooth)
    vwnd_850 = mpcalc.smooth_n_point(vwnd_data.metpy.sel(time=date, vertical=850*units.hPa), 9, smooth)
    tmpc_850 = (mpcalc.smooth_n_point(tmpk_data.metpy.sel(time=date, vertical=850*units.hPa), 9, smooth)).metpy.convert_units('degC')
    tmpc_700 = (mpcalc.smooth_n_point(tmpk_data.metpy.sel(time=date, vertical=700*units.hPa), 9, smooth)).metpy.convert_units('degC')
    relh_700 = mpcalc.smooth_n_point(relh_data.metpy.sel(time=date, vertical=700*units.hPa), 9, smooth)
    hght_300 = mpcalc.smooth_n_point(hght_data.metpy.sel(time=date, vertical=300*units.hPa), 9, smooth)
    uwnd_300 = mpcalc.smooth_n_point(uwnd_data.metpy.sel(time=date, vertical=300*units.hPa), 9, smooth)
    vwnd_300 = mpcalc.smooth_n_point(vwnd_data.metpy.sel(time=date, vertical=300*units.hPa), 9, smooth)
    
    mslp = (mpcalc.smooth_n_point(data.Pressure_reduced_to_MSL_msl.metpy.sel(time=date), 9, smooth)).metpy.convert_units('hPa')
    uwnd10 = mpcalc.smooth_n_point(data['u-component_of_wind_height_above_ground'].metpy.sel(time=date, vertical=10*units.meter), 9, smooth)
    vwnd10 = mpcalc.smooth_n_point(data['v-component_of_wind_height_above_ground'].metpy.sel(time=date, vertical=10*units.meter), 9, smooth)
    tmpk2 = (mpcalc.smooth_n_point(data.Temperature_height_above_ground.metpy.sel(time=date, vertical=2*units.meter), 9, smooth))
    tmpf2 = tmpk2.metpy.convert_units('degF')
    pcpin = hght_1000 * 0.0

    # Calculate Absolute Vorticity scale by 1e5
    absvort_500 = mpcalc.absolute_vorticity(uwnd_500, vwnd_500)*1e5

    # Calculate wind speed and convert units to knots
    wspd_300 = mpcalc.wind_speed(uwnd_300, vwnd_300).metpy.convert_units('kts')
    
    # To set the map area, need to convert to proper coords.
    LL = plotcrs.transform_point(-125., 22., ccrs.PlateCarree())
    UR = plotcrs.transform_point(-55., 52., ccrs.PlateCarree())
    
    def plot_background(num, crs1):
        ax = plt.subplot(num, projection=crs1)
        #   ax.set_extent([west long, east long, south lat, north lat])
        ax.set_extent([LL[0], UR[0], LL[1], UR[1]], crs1)
        ax.add_feature(cfeature.COASTLINE.with_scale('50m'), edgecolor='black', linewidth=0.5)
        ax.add_feature(cfeature.STATES.with_scale('50m'), edgecolor='black', linewidth=0.5)
        ax.add_feature(cfeature.BORDERS.with_scale('50m'), edgecolor='black', linewidth=0.75)
        return ax

    # Set contour intervals
    clevpmsl = np.arange(800, 1100, 4)
    clev850 = np.arange(0, 5000, 30)
    clevrh700 = [50, 70, 80, 90, 100]
    clevtmpc850 = np.arange(-50, 51, 2)
    clev500 = np.arange(0, 10000, 60)
    clev300 = np.arange(0, 15000, 120)
    clevsped300 = np.arange(50, 230, 20)
    clevprecip = [0, 0.01, 0.03, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.40, 0.50,
                  0.60, 0.70, 0.80, 0.90, 1.00, 1.25, 1.50, 1.75, 2.00, 2.50]
    clevavor500 = list(range(-8, 1, 1))+list(range(8, 49, 2))
    colors1 = plt.cm.YlOrRd(np.linspace(0, 1, 31))
    colors2 = plt.cm.BuPu(np.linspace(0.5, 0.75, 8))
    colorsavor500 = np.vstack((colors2, (1, 1, 1, 1), colors1))
    clevtmpf2 = np.arange(-60, 120, 5)

    # Use simple scheme for thinning wind barbs on plots
    dlon = lon[1] - lon[0]
    if dlon == 1:
        barb_spacing = 3
    elif dlon == 0.5:
        barb_spacing = 6
    elif dlon == 0.25:
        barb_spacing = 10
    else:
        barb_spacing = 2
    wind_slice = (slice(None, None, barb_spacing), slice(None, None, barb_spacing))

    # Create 4-panel plot
    fig=plt.figure(1, figsize=(22., 13.))
    fig.subplots_adjust(top=1, bottom=0, hspace=0.0, wspace=0.01)
    
    # Upper-left panel MSLP, 1000-500 hPa Thickness, Precip (in)
    ax1 = plot_background(221, plotcrs)
    ax1.set_extent([230., 290., 20., 55.], ccrs.PlateCarree())
    plt.title('GFS - MSLP (hPa), 2m TMPF, and 1000-500 hPa Thickness (m)', loc='left')
    plt.title(f'VALID: {dt}', loc='right')
    cmap = plt.cm.coolwarm
    cf = ax1.contourf(tlon, tlat, tmpf2, clevtmpf2, cmap=cmap, extend='both')
    cbar = plt.colorbar(cf, orientation='horizontal', extend='both', aspect=65, pad=0, extendrect='True')
    ax1.barbs(lons[wind_slice], lats[wind_slice],
              uwnd10.metpy.convert_units('kts').values[wind_slice],
              vwnd10.metpy.convert_units('kts').values[wind_slice],
              length=6, transform=datacrs)
    cs2 = ax1.contour(tlon, tlat, hght_500-hght_1000, clev500, colors='grey', linewidths=1.5, linestyles='dashed')
    cs  = ax1.contour(tlon, tlat, mslp, clevpmsl, colors='black', linewidths=1.5)
    plt.clabel(cs, fontsize=10, inline=True, fmt='%d', rightside_up=True)
    plt.clabel(cs2, fontsize=9, inline=True, fmt='%d', rightside_up=True)
        
    # Upper-right panel 850-hPa Heights and Temp (C)
    ax2 = plot_background(222, plotcrs)
    ax2.set_extent([230., 290., 20., 55.], ccrs.PlateCarree())
    cmap = plt.cm.Greens
    cf = ax2.contourf(tlon, tlat, relh_700.metpy.convert_units('percent'),
                      clevrh700, cmap=cmap, extend='both', norm=plt.Normalize(50, 120))
    cbar = plt.colorbar(cf, orientation='horizontal', extend='both', aspect=65, pad=0, extendrect='True')
    cs1 = ax2.contour(tlon, tlat, tmpc_850, clevtmpc850, colors='tab:red', linewidths=1.5, linestyles=':', alpha=0.75)
    cs = ax2.contour(tlon, tlat, hght_850, clev850, colors='black', linewidths=1.5)
    ax2.barbs(lons[wind_slice], lats[wind_slice],
              uwnd_850.metpy.convert_units('kts').values[wind_slice],
              vwnd_850.metpy.convert_units('kts').values[wind_slice],
              length=6, transform=datacrs)
    plt.clabel(cs, fontsize=10, inline=True, fmt='%d', rightside_up=True)
    plt.clabel(cs1, fontsize=10, inline=True, fmt='%d')
    plt.title('GFS - 850-hPa HGHTs (m) and TMPC', loc='left')
    plt.title(f'VALID: {dt}', loc='right')
        
    # Lower-left panel 500-hPa Heights and AVOR
    ax3 = plot_background(223, plotcrs)
    ax3.set_extent([230., 290., 20., 55.], ccrs.PlateCarree())
    cf = ax3.contourf(tlon, tlat, absvort_500, clevavor500, colors=colorsavor500, extend='both')
    cs1 = ax3.contour(tlon, tlat, absvort_500, clevavor500, colors='black', linewidths=1.5, linestyles=':', alpha=0.5)
    cbar = plt.colorbar(cf, orientation='horizontal', extend='both', aspect=65, pad=0, extendrect='True')
    cs = ax3.contour(tlon, tlat, hght_500[:,:], clev500, colors='black', linewidths=1.5)
    ax3.barbs(lons[wind_slice], lats[wind_slice],
              uwnd_500.metpy.convert_units('kts').values[wind_slice],
              vwnd_500.metpy.convert_units('kts').values[wind_slice],
              length=6, transform=datacrs)
    plt.clabel(cs, fontsize=10, inline=True, fmt='%d', rightside_up=True)
    plt.title('GFS - 500-hPa HGHTs (m) and AVOR ($*10^5$ $s^{-1}$)', loc='left')
    plt.title(f'VALID: {dt}', loc='right')
        
    # Lower-right panel 300-hPa Heights and Wind Speed (kts)
    ax4 = plot_background(224, plotcrs)
    ax4.set_extent([230., 290., 20., 55.], ccrs.PlateCarree())
    cmap = plt.cm.BuPu
    cf = ax4.contourf(tlon, tlat, wspd_300, clevsped300, cmap=cmap, extend='max')
    cs1 = ax4.contour(tlon, tlat, wspd_300, clevsped300, colors='black', linestyles=':', alpha=0.5)
    cbar = plt.colorbar(cf, orientation='horizontal', extend='max', aspect=65, pad=0, extendrect='True')
    cs = ax4.contour(tlon, tlat, hght_300, clev300, colors='k', linewidths=1.5)
    ax4.barbs(lons[wind_slice], lats[wind_slice],
              uwnd_300.metpy.convert_units('kts').values[wind_slice],
              vwnd_300.metpy.convert_units('kts').values[wind_slice],
              length=6, transform=datacrs)
    plt.clabel(cs, fontsize=10, inline=True, fmt='%d', rightside_up=True)
    plt.title('GFS - 300-hPa HGHTs (m) and SPED (kts)', loc='left')
    plt.title(f'VALID: {date}', loc='right')
    
    if save:
        plt.savefig(f'GFS_{date:%Y%m%d}_{date:%H00}.png', dpi=150, bbox_inches='tight')
        plt.close()
    else:
        plt.show()



In [ ]:
year_widget = widgets.Dropdown(
    options=list(range(1999, datetime.utcnow().year+1)),
    description='Year', alignment='center', value=2015)
month_widget = widgets.Dropdown(
    options=list(range(1, 13)),
    description='Month', alignment='center', value=1)
day_widget = widgets.Dropdown(description='Day', options=list(range(1, 32)), value=1)
hour_widget = widgets.Dropdown(description='Hour', options=[0, 6, 12, 18], value=0)


In [ ]:
x = widgets.interact_manual(getdata, year=year_widget, month=month_widget, day=day_widget, hour=hour_widget)